# Noves variables

In [1]:
import pandas as pd # Data handling and managing
import numpy as np  # Handiling linear Algera
import seaborn as sn
import datetime 
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import xgboost as xgb #xgboost model
import pmdarima as pm

%matplotlib inline

In [25]:
dataset=pd.read_excel(r'C:\Users\subir\OneDrive\Escritorio\PC ANTIC\UOC\TFM\Dataset_Completo_Fase2.xlsx')
dataset.drop(dataset.columns[0], axis=1, inplace=True)
dataset

,Fecha,Producto,udsVenta,flag,Precio,Festivo,Fin de semana
0,2015-06-29,1,3629,0,4.394742,0,0
1,2015-06-30,1,4666,0,4.394742,0,0
2,2015-07-01,1,16070,0,4.394742,0,0
3,2015-07-02,1,4666,0,4.394742,0,0
4,2015-07-03,1,8813,0,4.394742,0,0
...,...,...,...,...,...,...,...
140541,2021-03-19,99,540,0,4.394742,0,0
140542,2021-03-20,99,0,0,4.394742,0,1
140543,2021-03-21,99,0,0,4.394742,0,1
140544,2021-03-22,99,0,0,4.394742,0,0


In [26]:
Q1_ventas=dataset['udsVenta'].quantile(0.25)
Q3_ventas=dataset['udsVenta'].quantile(0.75)
IQR_ventas=Q3_ventas-Q1_ventas

Limit_lower=Q1_ventas-(1.5*IQR_ventas)
Limit_upper=Q3_ventas+(1.5*IQR_ventas)

dataset=dataset[~((dataset["udsVenta"] < Limit_lower) |(dataset["udsVenta"] > Limit_upper))]

In [27]:
# sort the values per store, item and date so that values would be shifted equally

dataset.sort_values(by=['Producto', 'Fecha'], axis=0, inplace=True)

<ipython-input-27-a3a0f396d86f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.sort_values(by=['Producto', 'Fecha'], axis=0, inplace=True)


In [28]:
# the feature name will be created dynamically with regards to the lag value for a given list of lags

def lag_features(dataframe, lags):
    dataframe = dataframe.copy()
    for lag in lags:
        dataframe['sales_lag_' + str(lag)] = dataframe.groupby(["Producto"])['udsVenta'].transform(
            lambda x: x.shift(lag))
    return dataframe

In [29]:
dataset = lag_features(dataset, [7,15,30,60,90,120, 150, 180, 210, 240, 270, 300, 330,360])

In [30]:
dataset

,Fecha,Producto,udsVenta,flag,Precio,Festivo,Fin de semana,sales_lag_7,sales_lag_15,sales_lag_30,...,sales_lag_90,sales_lag_120,sales_lag_150,sales_lag_180,sales_lag_210,sales_lag_240,sales_lag_270,sales_lag_300,sales_lag_330,sales_lag_360
5,2015-07-04,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-07-05,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2015-07-11,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2015-07-12,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2015-07-18,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140541,2021-03-19,99,540,0,4.394742,0,0,0.0,0.0,756.0,...,0.0,0.0,162.0,0.0,0.0,0.0,432.0,0.0,0.0,0.0
140542,2021-03-20,99,0,0,4.394742,0,1,0.0,0.0,0.0,...,0.0,540.0,648.0,0.0,0.0,0.0,0.0,540.0,0.0,108.0
140543,2021-03-21,99,0,0,4.394742,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1620.0,0.0,0.0,0.0,0.0
140544,2021-03-22,99,0,0,4.394742,0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1188.0,0.0,810.0,0.0,0.0


In [31]:
def roll_mean_features(dataframe, windows):
    dataframe = dataframe.copy()
    for window in windows:
        dataframe['sales_roll_mean_' + str(window)] = dataframe.groupby(["Producto"])['udsVenta']. \
                                                          transform(
            lambda x: x.shift(1).rolling(window=window, min_periods=10, win_type="triang").mean()) 
    return dataframe

In [32]:
dataset = roll_mean_features(dataset, [365, 546, 730])
dataset.head()

,Fecha,Producto,udsVenta,flag,Precio,Festivo,Fin de semana,sales_lag_7,sales_lag_15,sales_lag_30,...,sales_lag_180,sales_lag_210,sales_lag_240,sales_lag_270,sales_lag_300,sales_lag_330,sales_lag_360,sales_roll_mean_365,sales_roll_mean_546,sales_roll_mean_730
5,2015-07-04,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-07-05,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2015-07-11,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,2015-07-12,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2015-07-18,1,0,0,4.394742,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
dataset['Mes']=pd.DatetimeIndex(dataset['Fecha']).month

In [34]:
dataset['Dia']=pd.DatetimeIndex(dataset['Fecha']).day

In [35]:
dataset['DiaDeLaSemana']=pd.DatetimeIndex(dataset['Fecha']).dayofweek

In [36]:
dataset['Año']=pd.DatetimeIndex(dataset['Fecha']).year

In [37]:
dataset=dataset.dropna()
dataset

,Fecha,Producto,udsVenta,flag,Precio,Festivo,Fin de semana,sales_lag_7,sales_lag_15,sales_lag_30,...,sales_lag_300,sales_lag_330,sales_lag_360,sales_roll_mean_365,sales_roll_mean_546,sales_roll_mean_730,Mes,Dia,DiaDeLaSemana,Año
1334,2019-02-20,1,1555,0,4.138200,0,0,0.0,1555.0,518.0,...,0.0,0.0,0.0,121.425704,94.570180,86.954522,2,20,2,2019
1337,2019-02-23,1,1037,0,4.138200,0,1,1037.0,1037.0,0.0,...,0.0,0.0,0.0,122.785716,95.307368,87.471774,2,23,5,2019
1338,2019-02-24,1,0,0,4.138200,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,124.180002,96.063975,88.001770,2,24,6,2019
1341,2019-02-27,1,1037,0,4.138200,0,0,518.0,1037.0,0.0,...,0.0,0.0,0.0,125.577704,96.826398,88.532477,2,27,2,2019
1344,2019-03-02,1,518,0,4.138200,0,1,0.0,0.0,1037.0,...,0.0,0.0,0.0,127.009914,97.594623,89.063859,3,2,5,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140541,2021-03-19,99,540,0,4.394742,0,0,0.0,0.0,756.0,...,0.0,0.0,0.0,198.518439,218.510439,201.719088,3,19,4,2021
140542,2021-03-20,99,0,0,4.394742,0,1,0.0,0.0,0.0,...,540.0,0.0,108.0,197.608648,218.441607,201.748880,3,20,5,2021
140543,2021-03-21,99,0,0,4.394742,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,196.698856,218.366616,201.782522,3,21,6,2021
140544,2021-03-22,99,0,0,4.394742,0,0,0.0,0.0,0.0,...,810.0,0.0,0.0,195.789065,218.287277,201.817988,3,22,0,2021


In [38]:
dataset.to_excel(r'C:\Users\subir\OneDrive\Escritorio\PC ANTIC\UOC\TFM\Dataset_Completo_Fase3.xlsx')